In [17]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib
import geopandas as gp
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points
import Outils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [ ]:
toto=at.affecter_troncon_ligne('TRONROUT0000000109674477')

In [ ]:
toto

In [111]:
dico=at.affecter_troncon(df)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000113489744 à 17:10:17 nb ligne traite : 0, nb ligne differente=0


C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x), axis=1)


300eme occurence : TRONROUT0000000129676524 à 17:10:53 nb ligne traite : 846, nb ligne differente=846


IndexError: index 0 is out of bounds for axis 0 with size 0

pour nouvelle fonction de loc des route à 2 chaussées

In [7]:
#liste des troncons (parametre entrant)
liste_troncon=['TRONROUT0000000130923326','TRONROUT0000000109674477','TRONROUT0000000032839287','TRONROUT0000000032834970','TRONROUT0000000032837041','TRONROUT0000000032837045','TRONROUT0000000032839277','TRONROUT0000000032839282','TRONROUT0000000032839286','TRONROUT0000000032841957']

In [13]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom') 

In [11]:
#union des geometries
gdf_lignes2=gdf_lignes.unary_union

In [81]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.centroid.buffer(200).bounds

In [50]:
#donnees de base
gdf_global=gp.GeoDataFrame(df, geometry='geom')

In [82]:
#recherche des lignes proches du centroid
lignes_possibles=gdf_global.cx[xmin:xmax,ymin:ymax]

In [83]:
lignes_possibles

,id,prec_plani,prec_alti,nature,numero,nom_voie_g,nom_voie_d,importance,cl_admin,gestion,...,long_km,source,target,np_ref,geom,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom
id_ign,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000032839277,132722,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,0.428882,572,2942,None,(LINESTRING (414099.0999999996 6573414.8999999...,1.0,2,POINT(414099.1 6573414.89999999),2,POINT(413714.1 6573603.79999999)
TRONROUT0000000032839278,132724,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,1.087204,2943,1126,None,(LINESTRING (413677.9999999997 6573637.7999999...,3.0,2,POINT(413678 6573637.79999999),2,POINT(412700.9 6574114.49999999)
TRONROUT0000000032839279,132725,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,1.647285,3026,3027,None,(LINESTRING (413686.9999999997 6573633.5999999...,3.0,2,POINT(413687 6573633.59999999),2,POINT(415161.7 6572900.29999999)
TRONROUT0000000032839282,132727,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,1.086388,3048,2899,None,(LINESTRING (413672.5999999996 6573624.0999999...,1.0,2,POINT(413672.6 6573624.09999999),2,POINT(412696.6 6574101.19999999)
TRONROUT0000000032839288,132730,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,0.009932,3026,2943,None,(LINESTRING (413686.9999999997 6573633.5999999...,3.0,2,POINT(413687 6573633.59999999),2,POINT(413678 6573637.79999999)
TRONROUT0000000032839286,132729,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,0.037836,2942,3102,None,(LINESTRING (413714.0999999996 6573603.7999999...,1.0,2,POINT(413714.1 6573603.79999999),2,POINT(413680.1 6573620.39999999)
TRONROUT0000000032839287,88358,1.5,1.0,Quasi-autoroute,N11,None,None,1,Nationale,DA,...,0.008363,3048,3102,None,(LINESTRING (413672.5999999996 6573624.0999999...,1.0,2,POINT(413672.6 6573624.09999999),2,POINT(413680.1 6573620.39999999)


In [84]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles.loc[:,'numero']==gdf_lignes.iloc[0]['numero'])]

In [110]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [102]:
#garder uniquement la valeur la plus proche du centroid
ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index[0]

'TRONROUT0000000032839278'